In [19]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=17)

def parses(data):
    registers, program =  data.strip().split('\n\n')
    registers = [int(r.split(': ')[-1]) for r in registers.split('\n')]
    program = [int(i) for i in program.strip().split(': ')[-1].split(',')]
    return registers, program
    

data = parses(puzzle.input_data)

In [67]:
sample = parses("""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0""")

In [79]:
from parse import parse

template = """Register A: {:d}
Register B: {:d}
Register C: {:d}

Program: {}"""

def parses(data):
    a, b, c, program = parse(template, data).fixed
    program = [int(x) for x in program.split(",")]
    return (a, b, c), program

def run_computer(registers, program):
    a, b, c = registers
    ip = 0
    output = []
    
    def combo(op):
        if op <= 3:
            return op
        if op <= 6:
            return [a,b,c][op-4]
        raise ValueError("Invalid combo operand 7")
        
    while ip < len(program) - 1:
        op, val = program[ip:ip + 2]
        
        if op == 0:  # adv
            a = a >> combo(val)
        elif op == 1:  # bxl
            b ^= val  
        elif op == 2:  # bst
            b = combo(val) & 7
        elif op == 3:  # jnz
            ip = val if a else ip + 2
        elif op == 4:  # bxc
            b ^= c
        elif op == 5:  # out
            output.append(combo(val) & 7)
        elif op == 6:  # bdv
            b = a >> combo(val)
        elif op == 7:  # cdv  
            c = a >> combo(val)
        if op != 3: # no jump
            ip += 2
            
    return output

def solve_a(data):
    return ','.join(str(x) for x in run_computer(*data))

In [83]:
solve_a(sample)

'4,6,3,5,6,3,5,2,1,0'

In [110]:
def solve_b(data):
    _, program = data
    
    def to_decimal(seq):
        return sum(d * (8 ** i) for i, d in enumerate(reversed(seq)))

    # Search for solution using backtracking
    stack = [[i] for i in range(8)]
    
    while stack:
        seq = stack.pop()
        a = to_decimal(seq)
        output = run_computer((a, 0, 0), program)
        
        if output == program:
            return a
        
        # If current output matches target suffix, extend sequence
        n = len(output)
        if output[-n:] == program[-n:]:
            # reversed is important so we dfs smaller numbers first
            for i in reversed(range(8)): 
                stack.append(seq + [i])

In [111]:
sample_b = parses("""Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0""")

In [112]:
solve_b(sample_b)

117440

In [113]:
solve_b(data)

202991746427434

In [92]:
oct(117440)

'0o345300'

In [88]:
solve_b(data)

202991746427434

In [89]:
sample

((729, 0, 0), [0, 1, 5, 4, 3, 0])

In [87]:
solve_b(sample)

KeyboardInterrupt: 

In [ ]:
class ThreeBitComputer:
    def __init__(self, register_a=0, register_b=0, register_c=0):
        self.registers = {'A': register_a, 'B': register_b, 'C': register_c}
        self.instruction_pointer = 0
        self.output = []
        
    def get_combo_value(self, operand):
        """Resolve combo operand value according to rules"""
        if operand <= 3:
            return operand
        elif operand == 4:
            return self.registers['A']
        elif operand == 5:
            return self.registers['B']
        elif operand == 6:
            return self.registers['C']
        else:  # operand == 7
            raise ValueError("Invalid combo operand 7")

    def execute_instruction(self, opcode, operand):
        """Execute a single instruction based on opcode"""
        if opcode == 0:  # adv
            power = self.get_combo_value(operand)
            self.registers['A'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 1:  # bxl
            self.registers['B'] ^= operand
            return True
            
        elif opcode == 2:  # bst
            self.registers['B'] = self.get_combo_value(operand) % 8
            return True
            
        elif opcode == 3:  # jnz
            if self.registers['A'] != 0:
                self.instruction_pointer = operand
                return False
            return True
            
        elif opcode == 4:  # bxc
            self.registers['B'] ^= self.registers['C']
            return True
            
        elif opcode == 5:  # out
            value = self.get_combo_value(operand) % 8
            self.output.append(str(value))
            return True
            
        elif opcode == 6:  # bdv
            power = self.get_combo_value(operand)
            self.registers['B'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 7:  # cdv
            power = self.get_combo_value(operand)
            self.registers['C'] = self.registers['A'] // (2 ** power)
            return True
            
        else:
            raise ValueError(f"Invalid opcode: {opcode}")

    def run_program(self, program):
        """Run a program (list of integers) until halt"""
        # Convert string program to list of integers if needed
        if isinstance(program, str):
            program = [int(x.strip()) for x in program.split(',')]
            
        while self.instruction_pointer < len(program) - 1:
            opcode = program[self.instruction_pointer]
            operand = program[self.instruction_pointer + 1]
            
            # Execute instruction and check if we should advance instruction pointer
            should_advance = self.execute_instruction(opcode, operand)
            
            if should_advance:
                self.instruction_pointer += 2
                
        return ','.join(self.output)

# Example usage
def run_example(data):
    (register_a, register_b, register_c), program = data
    computer = ThreeBitComputer(register_a, register_b, register_c)
    result = computer.run_program(program)
    print(f"Output: {result}")
    return result

# # Test the examples from the problem
# if __name__ == "__main__":
#     # Test case from the problem
#     test_result = run_example(729, 0, 0, "0,1,5,4,3,0")
#     print(f"Test case output: {test_result}")
#     assert test_result == "4,6,3,5,6,3,5,2,1,0", "Test case failed!"
    
#     # Additional test cases from the problem description
#     computer = ThreeBitComputer(10, 0, 0)
#     assert computer.run_program("5,0,5,1,5,4") == "0,1,2"
    
#     computer = ThreeBitComputer(2024, 0, 0)
#     assert computer.run_program("0,1,5,4,3,0") == "4,2,5,6,7,7,7,7,3,1,0"

In [ ]:
def solve_a(data):
    pass

In [ ]:
solve_a(sample)

In [ ]:
solve_a(data)

In [ ]:
def solve_b(data):
    pass

In [ ]:
solve_b(sample)

In [ ]:
solve_b(data)

In [22]:
class ThreeBitComputer:
    def __init__(self, register_a=0, register_b=0, register_c=0):
        self.registers = {'A': register_a, 'B': register_b, 'C': register_c}
        self.instruction_pointer = 0
        self.output = []
        
    def get_combo_value(self, operand):
        """Resolve combo operand value according to rules"""
        if operand <= 3:
            return operand
        elif operand == 4:
            return self.registers['A']
        elif operand == 5:
            return self.registers['B']
        elif operand == 6:
            return self.registers['C']
        else:  # operand == 7
            raise ValueError("Invalid combo operand 7")

    def execute_instruction(self, opcode, operand):
        """Execute a single instruction based on opcode"""
        if opcode == 0:  # adv
            power = self.get_combo_value(operand)
            self.registers['A'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 1:  # bxl
            self.registers['B'] ^= operand
            return True
            
        elif opcode == 2:  # bst
            self.registers['B'] = self.get_combo_value(operand) % 8
            return True
            
        elif opcode == 3:  # jnz
            if self.registers['A'] != 0:
                self.instruction_pointer = operand
                return False
            return True
            
        elif opcode == 4:  # bxc
            self.registers['B'] ^= self.registers['C']
            return True
            
        elif opcode == 5:  # out
            value = self.get_combo_value(operand) % 8
            self.output.append(str(value))
            return True
            
        elif opcode == 6:  # bdv
            power = self.get_combo_value(operand)
            self.registers['B'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 7:  # cdv
            power = self.get_combo_value(operand)
            self.registers['C'] = self.registers['A'] // (2 ** power)
            return True
            
        else:
            raise ValueError(f"Invalid opcode: {opcode}")

    def run_program(self, program):
        """Run a program (list of integers) until halt"""
        # Convert string program to list of integers if needed
        if isinstance(program, str):
            program = [int(x.strip()) for x in program.split(',')]
            
        while self.instruction_pointer < len(program) - 1:
            opcode = program[self.instruction_pointer]
            operand = program[self.instruction_pointer + 1]
            
            # Execute instruction and check if we should advance instruction pointer
            should_advance = self.execute_instruction(opcode, operand)
            
            if should_advance:
                self.instruction_pointer += 2
                
        return ','.join(self.output)

# Example usage
def run_example(data):
    (register_a, register_b, register_c), program = data
    computer = ThreeBitComputer(register_a, register_b, register_c)
    result = computer.run_program(program)
    print(f"Output: {result}")
    return result

# # Test the examples from the problem
# if __name__ == "__main__":
#     # Test case from the problem
#     test_result = run_example(729, 0, 0, "0,1,5,4,3,0")
#     print(f"Test case output: {test_result}")
#     assert test_result == "4,6,3,5,6,3,5,2,1,0", "Test case failed!"
    
#     # Additional test cases from the problem description
#     computer = ThreeBitComputer(10, 0, 0)
#     assert computer.run_program("5,0,5,1,5,4") == "0,1,2"
    
#     computer = ThreeBitComputer(2024, 0, 0)
#     assert computer.run_program("0,1,5,4,3,0") == "4,2,5,6,7,7,7,7,3,1,0"

In [1]:
class ThreeBitComputer:
    def __init__(self, register_a=0, register_b=0, register_c=0):
        self.registers = {'A': register_a, 'B': register_b, 'C': register_c}
        self.instruction_pointer = 0
        self.output = []
        
    def get_combo_value(self, operand):
        """Resolve combo operand value according to rules"""
        if operand <= 3:
            return operand
        elif operand == 4:
            return self.registers['A']
        elif operand == 5:
            return self.registers['B']
        elif operand == 6:
            return self.registers['C']
        else:  # operand == 7
            raise ValueError("Invalid combo operand 7")

    def execute_instruction(self, opcode, operand, target_program):
        """Execute a single instruction based on opcode, with early exit for mismatches"""
        if opcode == 0:  # adv
            power = self.get_combo_value(operand)
            self.registers['A'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 1:  # bxl
            self.registers['B'] ^= operand
            return True
            
        elif opcode == 2:  # bst
            self.registers['B'] = self.get_combo_value(operand) % 8
            return True
            
        elif opcode == 3:  # jnz
            if self.registers['A'] != 0:
                self.instruction_pointer = operand
                return False
            return True
            
        elif opcode == 4:  # bxc
            self.registers['B'] ^= self.registers['C']
            return True
            
        elif opcode == 5:  # out
            value = self.get_combo_value(operand) % 8
            # Early exit if output doesn't match program
            if len(self.output) >= len(target_program):
                return None  # Signal to stop - too many outputs
            if value != target_program[len(self.output)]:
                return None  # Signal to stop - wrong output
            self.output.append(value)
            return True
            
        elif opcode == 6:  # bdv
            power = self.get_combo_value(operand)
            self.registers['B'] = self.registers['A'] // (2 ** power)
            return True
            
        elif opcode == 7:  # cdv
            power = self.get_combo_value(operand)
            self.registers['C'] = self.registers['A'] // (2 ** power)
            return True
            
        else:
            raise ValueError(f"Invalid opcode: {opcode}")

    def run_program(self, program, check_self_output=False):
        """Run a program (list of integers) until halt"""
        # Convert string program to list of integers if needed
        if isinstance(program, str):
            program = [int(x.strip()) for x in program.split(',')]
            
        target_program = program if check_self_output else None
            
        while self.instruction_pointer < len(program) - 1:
            opcode = program[self.instruction_pointer]
            operand = program[self.instruction_pointer + 1]
            
            # Execute instruction and check if we should advance instruction pointer
            should_advance = self.execute_instruction(opcode, operand, target_program)
            
            if should_advance is None:  # Early exit signal
                return False
                
            if should_advance:
                self.instruction_pointer += 2
        
        if check_self_output:
            return len(self.output) == len(program) and all(a == b for a, b in zip(self.output, program))
        return ','.join(map(str, self.output))

def find_lowest_self_output_a(program, start=1, step=1, verbose=False):
    """Find lowest value of A that makes the program output itself"""
    if isinstance(program, str):
        program = [int(x.strip()) for x in program.split(',')]
    
    a = start
    count = 0
    while True:
        computer = ThreeBitComputer(register_a=a, register_b=0, register_c=0)
        if computer.run_program(program, check_self_output=True):
            return a
        
        count += 1
        if count % 10000 == 0 and verbose:
            print(f"Tried A={a}")
        
        a += step

# Test with the example from the problem
if __name__ == "__main__":
    test_program = "2,4,1,1,7,5,4,4,1,4,0,3,5,5,3,0"
    result = find_lowest_self_output_a(test_program, verbose=True)
    print(f"\nFound solution: A={result}")
    
    # Verify the solution
    computer = ThreeBitComputer(register_a=result, register_b=0, register_c=0)
    output = computer.run_program(test_program)
    print(f"Program: {test_program}")
    print(f"Output: {output}")
    assert output == test_program.replace(" ", ""), "Solution verification failed!"

In [25]:

run_example(sample)

Output: 4,6,3,5,6,3,5,2,1,0


'4,6,3,5,6,3,5,2,1,0'

In [26]:
run_example(data)

Output: 7,4,2,0,5,0,5,3,7


'7,4,2,0,5,0,5,3,7'

In [ ]:
data